In [ ]:
import openai
import os
import sys
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

# Set your OpenAI API key saved in openai_files/api_key.txt
with open("openai_files/api_key.txt", "r") as f:
    openai.api_key = f.read().strip()
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = openai.api_key



sys.path.append("../../")
import biked_commons
from biked_commons.data_loading import data_loading
from biked_commons.conditioning import conditioning
from biked_commons.design_evaluation.design_evaluation import construct_tensor_evaluator, get_standard_evaluations
from biked_commons.transformation import one_hot_encoding

In [2]:
filedir = "openai_files"
os.makedirs(filedir, exist_ok=True)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
data = data_loading.load_bike_bench_mixed_modality_train()
data_sample = data.sample(10, random_state=42).reset_index(drop=True)

data_oh = one_hot_encoding.encode_to_continuous(data_sample)
data_tens = torch.tensor(data_oh.values, dtype=torch.float32).to(device)

In [4]:
def get_condition_by_idx(idx=0):
    rider_condition = conditioning.sample_riders(10, split="test")
    use_case_condition = conditioning.sample_use_case(10, split="test")
    text_embeddings = conditioning.sample_text(10, split="test")
    condition = {"Rider": rider_condition[idx], "Use Case": use_case_condition[idx], "Text": text_embeddings[idx]}
    return condition

def get_conditions_10k():
    rider_condition = conditioning.sample_riders(10000, split="test")
    use_case_condition = conditioning.sample_use_case(10000, split="test")
    text_embeddings = conditioning.sample_text(10000, split="test")
    conditions = {"Rider": rider_condition, "Use Case": use_case_condition, "Text": text_embeddings}
    return conditions


def build_text_condition(condition):

    rc = condition["Rider"]
    rc_text = f"Rider Body Dimensions: Upper leg length - {rc[0]}, Lower leg length - {rc[1]}, Arm length - {rc[2]}, Torso length - {rc[3]}, Neck and head length - {rc[4]}, Torso width - {rc[5]}"
    tc = condition["Text"]
    uc = condition["Use Case"]
    #get argmax of uc
    uc = uc.argmax()
    if uc == 0:
        uc_text = "Use Case: Road Biking"
    elif uc == 1:
        uc_text = "Use Case: Mountain Biking"
    elif uc == 2:
        uc_text = "Use Case: Commuting"
    tc_text = "Marketing Description: " + tc
    full_text = f"{rc_text}. {uc_text}. {tc_text}"
    return full_text

cond = get_condition_by_idx(0)
cond_text = build_text_condition(cond)

In [5]:
condition_dataset_path = os.path.join(filedir, "condition_dataset.txt")
score_dataset_path = os.path.join(filedir, "score_dataset.csv")
design_dataset_path = os.path.join(filedir, "design_dataset.csv")

prompt_1_path = os.path.join(filedir, "prompt1.txt")
prompt_2_path = os.path.join(filedir, "prompt2.txt")
prompt_3_path = os.path.join(filedir, "prompt3.txt")
prompt_3_uncond_path = os.path.join(filedir, "prompt3_uncond.txt")

criterion_descriptions_path = os.path.join(filedir, "criterion_descriptions.txt")
parameter_descriptions_path = os.path.join(filedir, "parameter_descriptions.txt")


In [6]:
ds_len = len(data_sample)

dataset_text_conditions = []
rider_condition = conditioning.sample_riders(ds_len, split="train", randomize=True)
use_case_condition = conditioning.sample_use_case(ds_len, split="train", randomize=True)
text_embeddings = conditioning.sample_text(ds_len, split="train", randomize=True)
for i in trange(ds_len):
    conditions = {"Rider": rider_condition[i], "Use Case": use_case_condition[i], "Text": text_embeddings[i]}
    dataset_text_conditions.append(build_text_condition(conditions))

with open(condition_dataset_path, "w") as f:
    for item in dataset_text_conditions:
        f.write("%s\n" % item)


100%|██████████| 10/10 [00:00<00:00, 10000.72it/s]


In [7]:
condition = {"Rider": rider_condition, "Use Case": use_case_condition, "Text": text_embeddings}
eval_fns = get_standard_evaluations(device, aesthetics_mode="Text")
evaluator, requirement_names, requirement_types = construct_tensor_evaluator(eval_fns, data_oh.columns)
eval_scores = evaluator(data_tens, condition)
eval_scores_df = pd.DataFrame(eval_scores.cpu().detach().numpy(), columns=requirement_names)
eval_scores_df.to_csv(score_dataset_path)

data_sample.to_csv(design_dataset_path)

In [8]:
import os
from openai import OpenAI
def query_openai(condition, cond_flag=False):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY") or "your-api-key-here")

    def load_file(filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            return f.read()

    PROMPT_1 = load_file(prompt_1_path)
    PROMPT_2 = load_file(prompt_2_path)
    PROMPT_3 = load_file(prompt_3_uncond_path)
    if cond_flag:
        PROMPT_3 = load_file(prompt_3_path)

    parameter_descriptions = load_file(parameter_descriptions_path)
    criterion_descriptions = load_file(criterion_descriptions_path)
    condition_dataset = load_file(condition_dataset_path)
    design_dataset = load_file(design_dataset_path)
    score_dataset = load_file(score_dataset_path)

    messages = [
        {"role": "user", "content": PROMPT_1},
        {"role": "user", "content": f"Parameter Descriptions:\n{parameter_descriptions}"},
        {"role": "user", "content": f"Criteria Descriptions:\n{criterion_descriptions}"},
        {"role": "user", "content": PROMPT_2},
        {"role": "user", "content": f"Condition Dataset:\n{condition_dataset}"},
        {"role": "user", "content": f"Design Dataset:\n{design_dataset}"},
        {"role": "user", "content": f"Score Dataset:\n{score_dataset}"},
        {"role": "user", "content": PROMPT_3},
        {"role": "user", "content": condition},
    ]

    return client.chat.completions.create(
        model="o4-mini-2025-04-16",
        messages=messages,
    )



In [9]:
from io import StringIO
def clean_code_block(text):
    if text.startswith("```csv"):
        text = text[len("```csv"):].strip()
    if text.startswith("```"):
        text = text[len("```"):].strip()
    if text.endswith("```"):
        text = text[:-3].strip()
    return text

def process_response_text_to_dataframe(response_text, expected_columns):
    cleaned_csv = clean_code_block(response_text)
    try:
        # Preprocess by splitting lines and truncating fields
        processed_lines = []
        for line_num, line in enumerate(cleaned_csv.strip().split('\n')):
            fields = line.split(',')
            if len(fields) > len(expected_columns):
                print(f"⚠️ Line {line_num + 1}: Truncating extra fields ({len(fields)} -> {len(expected_columns)})")
                fields = fields[:len(expected_columns)]
            processed_lines.append(','.join(fields))

        # Reconstruct the CSV string after truncation
        processed_csv = '\n'.join(processed_lines)

        # Load with pandas
        df_raw = pd.read_csv(StringIO(processed_csv), header=None, names=expected_columns)

        # Optional shape validation (can be removed if you trust the truncation)
        if df_raw.shape[1] != len(expected_columns):
            print(f"❌ Still mismatched column count: {df_raw.shape[1]} vs {len(expected_columns)}")
            return None

        return df_raw

    except Exception as e:
        print(f"❌ Failed to parse CSV response: {e}")
        return None



def save_raw_response(response, raw_log_path):
    with open(raw_log_path, "w", encoding="utf-8") as f:
        content = response.choices[0].message.content.strip()
        f.write(content + "\n")
    print(f"✅ Saved raw response to {raw_log_path}")

In [10]:
class condition_sampler():
    def __init__(self):
        conditions = get_conditions_10k()
        self.rider_condition = conditions["Rider"]
        self.use_case_condition = conditions["Use Case"]
        self.text_embeddings = conditions["Text"]

    def get_condition(self, indices):
        text_conditions = []
        for i, true_index in enumerate(indices):
            condition = {"Rider": self.rider_condition[true_index], "Use Case": self.use_case_condition[true_index], "Text": self.text_embeddings[true_index]}
            res = build_text_condition(condition)
            text_conditions.append(f"Condition {i+1}: {res} \n ")
        text_conditions = "".join(text_conditions)
        return text_conditions


In [26]:
from concurrent.futures import ThreadPoolExecutor, as_completed

cond_flag = True

if cond_flag:
    append = "cond"
    n_cond = 1
    n_batch = 1000
    cs = condition_sampler()
else:
    append = "uncond"
    n_cond = 10
    n_batch = 100

gen_dir = os.path.join(filedir, "generated_files", append)
os.makedirs(gen_dir, exist_ok=True)
os.makedirs(os.path.join(gen_dir, "raw"), exist_ok=True)
os.makedirs(os.path.join(gen_dir, "csvs"), exist_ok=True)

def process_single_batch(i, batch):
    print(f"🚀 Starting task for i={i}, batch={batch}")
    if cond_flag:
        indices = range(batch*10, (batch+1)*10)
        condition = cs.get_condition(indices)
    else:
        condition = build_text_condition(get_condition_by_idx(i))
    response_path = os.path.join(gen_dir, f"raw/_{i}_{batch}.txt")

    if os.path.exists(response_path):
        print(f"❌ Skipping existing response file: {response_path}")
    else:
        try:
            response = query_openai(condition, cond_flag=cond_flag)
            save_raw_response(response, response_path)
        except Exception as e:
            print(f"❌ Error querying OpenAI for i={i}, batch={batch}: {e}")
    # try:
    #load response from file
    with open(response_path, "r", encoding="utf-8") as f:
        raw_csv = f.read()
    print(f"📊 Processing response for i={i}, batch={batch}")
    df_all = process_response_text_to_dataframe(raw_csv, expected_columns=data_sample.columns)
    output_path = os.path.join(gen_dir, f"csvs/_{i}_{batch}.csv")
    df_all.to_csv(output_path, index=False)
    print("test")
    print(f"✅ Saved generated CSV to: {output_path}")
    # except Exception as e:
    #     print(f"❌ Error processing response for i={i}, batch={batch}: {e}")


# List of (i, batch) tasks to process
tasks = [(i, batch) for i in range(n_cond) for batch in range(n_batch)]  # Adjust ranges as needed

# Execute in parallel
with ThreadPoolExecutor(max_workers=24) as executor:  # Adjust worker count as needed
    futures = [executor.submit(process_single_batch, i, batch) for i, batch in tasks]
    for future in as_completed(futures):
        future.result()  # Will raise exceptions if any occurred inside the thread



🚀 Starting task for i=0, batch=0
🚀 Starting task for i=0, batch=1
🚀 Starting task for i=0, batch=2
🚀 Starting task for i=0, batch=3
🚀 Starting task for i=0, batch=4
🚀 Starting task for i=0, batch=5
🚀 Starting task for i=0, batch=6
🚀 Starting task for i=0, batch=7
🚀 Starting task for i=0, batch=8
🚀 Starting task for i=0, batch=9
🚀 Starting task for i=0, batch=10
🚀 Starting task for i=0, batch=11
🚀 Starting task for i=0, batch=12
🚀 Starting task for i=0, batch=13
🚀 Starting task for i=0, batch=14
🚀 Starting task for i=0, batch=15
🚀 Starting task for i=0, batch=16
🚀 Starting task for i=0, batch=17
🚀 Starting task for i=0, batch=18
❌ Skipping existing response file: openai_files\generated_files\cond\raw/_0_1.txt
🚀 Starting task for i=0, batch=19
❌ Skipping existing response file: openai_files\generated_files\cond\raw/_0_0.txt
🚀 Starting task for i=0, batch=20
🚀 Starting task for i=0, batch=21
❌ Skipping existing response file: openai_files\generated_files\cond\raw/_0_2.txt
🚀 Starting task 

In [27]:
import os
import pandas as pd

gen_dir = os.path.join(filedir, "generated_files", append)

def normalize_bool_series(series):
    """
    Maps accepted boolean forms to standard Python True or False.
    """
    def map_to_bool(value):
        value_str = str(value).strip().lower()
        if value_str in {"true", "1"}:
            return True
        elif value_str in {"false", "0"}:
            return False
        else:
            return value  # Leave unchanged if not recognized

    return series.apply(map_to_bool)

def is_castable_to_bool(series):
    """
    Checks if a pandas Series can be safely cast to boolean.
    Accepts 'True', 'False', '1', '0', 1, 0 (case insensitive).
    """
    valid_bool_values = {"true", "false", "1", "0", True, False, 1, 0}
    return series.apply(lambda x: str(x).strip().lower() in {"true", "false", "1", "0"})

def is_castable_column(series, target_dtype):
    if pd.api.types.is_bool_dtype(target_dtype):
        return is_castable_to_bool(series).all()
    elif pd.api.types.is_numeric_dtype(target_dtype):
        try:
            pd.to_numeric(series)
            return True
        except Exception:
            return False
    elif pd.api.types.is_datetime64_any_dtype(target_dtype):
        try:
            pd.to_datetime(series)
            return True
        except Exception:
            return False
    elif pd.api.types.is_object_dtype(target_dtype) or pd.api.types.is_string_dtype(target_dtype):
        return True
    else:
        try:
            series.astype(target_dtype)
            return True
        except Exception:
            return False

def get_valid_rows(df, reference_df):
    valid_mask = pd.Series([True] * len(df), index=df.index)  # Align mask with df index
    for col in df.columns:
        target_dtype = reference_df[col].dtype
        is_valid_col = df[col].apply(lambda x: is_castable_column(pd.Series([x]), target_dtype))
        valid_mask &= is_valid_col
    return df.loc[valid_mask]


for i in range(n_cond):  # Adjust as needed
    df_all = None
    for batch in range(n_batch):  # Adjust as needed

        
        output_path = os.path.join(gen_dir, f"csvs/_{i}_{batch}.csv")
        if not os.path.exists(output_path):
            print(f"❌ Skipping missing output file: {output_path}")
            continue

        # Read file assuming no header row
        df = pd.read_csv(output_path)

        #if df is longer than 10 rows, drop any extra
        if len(df) > 10:
            df = df.iloc[:10]

        indices = range(batch*10, batch*10+len(df))
        df.index = indices

        # Accumulate
        df_all = df if df_all is None else pd.concat([df_all, df])

    # Skip if nothing was loaded
    if df_all is None:
        print(f"⚠️  No valid batches for group {i}.")
        continue

    # Assign the correct column names
    df_all.columns = data_sample.columns

    # Type enforcement and filtering
    df_all_valid = get_valid_rows(df_all, data_sample)

    for col in df_all_valid.columns:
        if pd.api.types.is_bool_dtype(data_sample[col].dtype):
            df_all_valid[col] = normalize_bool_series(df_all_valid[col])


    # Save the cleaned DataFrame
    output_cleaned_path = os.path.join(gen_dir, f"cleaned_output_{i}.csv")
    df_all_valid.to_csv(output_cleaned_path)
    print(f"✅ Saved cleaned CSV to: {output_cleaned_path}")
    print(f"Valid designs: {len(df_all_valid)}")

    #print invalid designs
    invalid_designs = df_all[~df_all.index.isin(df_all_valid.index)]

C:\Users\Lyle\AppData\Local\Temp\ipykernel_40644\3957647327.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_valid[col] = normalize_bool_series(df_all_valid[col])


✅ Saved cleaned CSV to: openai_files\generated_files\cond\cleaned_output_0.csv
Valid designs: 9692


In [28]:
from biked_commons.design_evaluation.scoring import construct_scorer, MainScores, DetailedScores
from biked_commons.benchmark_models import benchmarking_utils
import importlib
importlib.reload(one_hot_encoding)
importlib.reload(benchmarking_utils)


if cond_flag:
    preds = pd.read_csv(os.path.join(gen_dir, f"cleaned_output_{0}.csv"), index_col=0)

    preds_oh = one_hot_encoding.encode_to_continuous(preds)
    #drop any rows with NaN values
    preds_oh = preds_oh.dropna()
    indices = preds_oh.index
    preds_tens = torch.tensor(preds_oh.values, dtype=torch.float32)
    main_scores, detailed_scores = benchmarking_utils.evaluate_cond(preds_tens, "O4-mini", preds_oh.columns, "cpu", indices=indices)
    print(main_scores)
else:
    for i in range(10):
        preds = pd.read_csv(os.path.join(gen_dir, f"cleaned_output_{i}.csv"), index_col=0)
        preds_oh = one_hot_encoding.encode_to_continuous(preds)
        preds_oh = preds_oh.dropna()
        preds_tens = torch.tensor(preds_oh.values, dtype=torch.float32)
        main_scores, detailed_scores = benchmarking_utils.evaluate_uncond(preds_tens, "O4-mini", i, preds_oh.columns, "cpu")
        print(main_scores)

⚠️ Warning: Column 'MATERIAL' has unknown values: {'False', 'True'}
⚠️ Warning: Column 'RIM_STYLE front' has unknown values: {'SPKED'}
⚠️ Warning: Column 'RIM_STYLE rear' has unknown values: {'SPKED'}
⚠️ Warning: Column 'Handlebar style' has unknown values: {'3'}
⚠️ Warning: Column 'Stem kind' has unknown values: {'3'}
⚠️ Warning: Column 'Seat tube type' has unknown values: {'43', 'nan', '3', '34.9', '26.8'}
Hypervolume                     0.261473
Constraint Satisfaction Rate    0.006274
Maximum Mean Discrepancy        0.346000
dtype: float64
